# Como Graficar Vectores de mas de 3 Dimensiones

In [ ]:
!pip install -U sentence-transformers --quiet

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
#Textos a vectorizar
sentences = [
    'Italy',
    'Spain',
    'Germany',
    'France'
]

In [ ]:
#Las frases son vectorizadas llamando a model.encode()
embeddings = model.encode(sentences)

In [ ]:
query = "Madrid"
query_embedding = model.encode(query)

## Dibujar los vectores

### Projectar los vectores en espacios de 1, 2 o 3 dimesiones

Hay tecnicas para graficar vectores que tienen mas de 3 dimensiones, por ejemplo PCA, t-sne y umap. Estas tecnicas projectan estos vectores de muchas dimensiones en espacios de menos dimensiones para poderlos graficar.

A continuacion veremos como implementar [UMAP](https://umap-learn.readthedocs.io/en/latest/index.html), para ello primero instalamos e importamos la libreria.

In [ ]:
!pip install tsne -U -q

In [ ]:
import tsne

Ahora utilizamos UMAP para projectar los vectores de 348 dimensiones a solo dos dimensiones

In [ ]:
umap_transform = umap.UMAP().fit(embeddings)

projected_dataset_embeddings = umap_transform.transform(embeddings)

projected_query_embedding = umap_transform.transform([query_embedding])

Una vez que hemos reducido las dimensiones de nuestro espacio procedemos a graficar con matplotlib como siempre

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()
ax.scatter(projected_dataset_embeddings[:, 0], projected_dataset_embeddings[:, 1], s=10)
for i, txt in enumerate(projected_dataset_embeddings):
    ax.annotate(sentences[i], (projected_dataset_embeddings[i,0], projected_dataset_embeddings[i,1]))
ax.scatter(projected_query_embedding[:, 0], projected_query_embedding[:, 1], c='red')
ax.annotate("Madrid", (projected_query_embedding[:, 0], projected_query_embedding[:, 1]))
ax.set_title('Vectores')
ax.axis('off');

### Utilizar de grafos para mostrar distancias entre vectores

A continuacion utilizaremos la libreria [networkx](https://networkx.org/) para graficar las distancias entre vectores como un grafo donde los nodos representan los datos (vectores) y las conecciones representan la distancia entre dichos datos.

In [ ]:
distancias = []
# Calcular que vector del chat es mas similar al vector a buscar
for i in range(len(embeddings)):
    cos_sim = util.cos_sim(query_embedding, embeddings[i,:])
    distancias.append(( 1 - float(cos_sim), sentences[i]))
    
distancias.sort(key=lambda tup: -tup[0])
distancias

In [ ]:
!pip install networkx -U -q

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
G = nx.Graph()
G.add_edge('Madrid', 'Italy', weight=0.42 )
G.add_edge('Madrid', 'Spain', weight=0.26 )
G.add_edge('Madrid', 'Germany', weight=0.47 )
G.add_edge('Madrid', 'France', weight=0.45 )

In [ ]:
pos = nx.spring_layout(G, seed=42)
nx.draw_networkx_nodes(G, pos, node_size=700)
nx.draw_networkx_edges(G, pos, width=2)

nx.draw_networkx_labels(G, pos, font_size=20, font_family="sans-serif")
edge_labels = nx.get_edge_attributes(G, "weight")
nx.draw_networkx_edge_labels(G, pos, edge_labels)

ax = plt.gca()
ax.margins(0.08)
plt.axis("off")
plt.tight_layout()
plt.show();

### Referencias:
- NetworkX: 
  - [Tutorial](https://networkx.org/documentation/stable/tutorial.html)
  - [Weighted Graph](https://networkx.org/documentation/stable/auto_examples/drawing/plot_weighted_graph.html)
  - [Node Position](https://networkx.org/documentation/stable/auto_examples/drawing/plot_center_node.html)

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/deeplearning/deeplearning.html)